# Import Library

In [1]:
# 기본 패키지
import pandas as pd
import numpy as np
import os

# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='NanumBarunGothic') 
plt.style.use(['ggplot'])

# 모델링 패키지
import sklearn
import matplotlib
# import pycaret
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import validation_curve
from sklearn.metrics import mean_squared_log_error

# 경고 메시지 무시
import warnings
warnings.filterwarnings(action='ignore') 

# 사용자 운영체제 확인
import platform
platform.system()

# 운영체제별 한글 폰트 설정
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')

plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정


# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'

# 버전 확인
print('Pandas : %s'%(pd.__version__))
print('Numpy : %s'%(np.__version__))
print('Scikit-Learn : %s'%(sklearn.__version__))
print('seaborn : %s'%(sns.__version__))
print('matplotlib : %s'%(matplotlib.__version__))
# print('pycaret : %s'%(pycaret.__version__))
!python --version

Pandas : 1.4.2
Numpy : 1.19.5
Scikit-Learn : 0.23.2
seaborn : 0.11.2
matplotlib : 3.5.2
Python 3.9.12


# Load Data

In [2]:
# Data Load
demo = pd.read_csv('../data/LPOINT_BIG_COMP_01_DEMO.csv', low_memory=False)
pdde = pd.read_csv('../data/LPOINT_BIG_COMP_02_PDDE.csv', low_memory=False)
cop = pd.read_csv('../data/LPOINT_BIG_COMP_03_COP_U.csv', low_memory=False)
clac = pd.read_csv('../data/LPOINT_BIG_COMP_04_PD_CLAC.csv', low_memory=False)
br = pd.read_csv('../data/LPOINT_BIG_COMP_05_BR.csv', low_memory=False)
lpay = pd.read_csv('../data/LPOINT_BIG_COMP_06_LPAY.csv', low_memory=False)
covid19 = pd.read_csv("../data/[한국] 대한민국의 코로나19 총 누적 확진자와 사망자 _ 일간 (2022년 8월 9일).csv", low_memory = False)

# Column명 변경
demo.columns = ['고객번호', '성별', '연령대', '거주지대분류코드']
pdde.columns = ['고객번호', '영수증번호', '채널구분', '제휴사', '점포코드', '상품코드', '구매일자', '구매시간', '구매금액', '구매수량']
cop.columns = ['고객번호', '영수증번호', '제휴사', '점포코드', '채널구분', '이용일자', '방문일자', '이용시간', '이용금액']
clac.columns = ['상품코드', '소분류명', '대분류명', '중분류명']
br.columns = ['점포코드', '제휴사', '점포대분류코드', '점포중분류코드']
lpay.columns = ['고객번호', '영수증번호', '제휴사', '채널구분', '이용일자', '이용시간', '이용금액']

####  [2. pdde]: 상품 구매 정보

In [3]:
# 중복되는 행 제거 -> 데이터 추출시 고객, 상품 각각 추출하여 중복이 발생한 것으로 판단함
pdde.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)

# 결측치 치환 -> na값을 온라인으로 채워줌
pdde['점포코드'] = pdde['점포코드'].fillna('온라인')

#### [3. cop]: 제휴사 이용 정보

In [4]:
# 결측치 치환
cop['점포코드'] = cop['점포코드'].fillna('온라인')

#### [외부데이터]: 일별 코로나 확진자 수

In [5]:
covid19 = covid19[['일자', '신규확진(명)', '신규사망(명)']]
covid19.columns = ['날짜', '신규확진자', '신규사망자']
covid19['날짜'] = covid19['날짜'].apply(lambda x: x.replace('/', '-')).astype(str)
covid19['날짜'] = pd.to_datetime(covid19['날짜'])
covid19 = covid19.sort_values(by = '날짜')
covid19 = covid19.reset_index(drop=True)
# covid19['현재격리자'] = 0
covid19

,날짜,신규확진자,신규사망자
0,2021-01-01,824,25
1,2021-01-02,651,20
2,2021-01-03,1020,19
3,2021-01-04,715,26
4,2021-01-05,839,20
...,...,...,...
581,2022-08-05,110610,45
582,2022-08-06,105507,27
583,2022-08-07,55292,29
584,2022-08-08,149819,40


# Data Merge

In [6]:
# 데이터를 통합하기 위하여 칼럼 통합
pdde['타입'] = 'Product'
cop['타입'] = 'Affiliate'
lpay['타입'] = 'Lpay'
pdde['방문일자'] = 0
lpay['방문일자'] = 0
cop['구매수량'] = 0
cop['상품코드'] = np.nan
lpay['점포코드'] = np.nan
lpay['상품코드'] = np.nan
lpay['구매수량'] = 0

In [7]:
# 칼럼명을 새롭게 동일하게 통일
cop.columns = ['고객번호', '영수증번호', '제휴사', '점포코드', '채널구분', '구매일자', '방문일자', '구매시간', '구매금액', '타입', '구매수량', '상품코드']
lpay.columns = ['고객번호', '영수증번호', '제휴사', '채널구분', '구매일자', '구매시간', '구매금액', '타입', '방문일자', '점포코드', '상품코드', '구매수량']

# 상품구매데이터를 기준으로 하여 열을 통일
cop = cop[['고객번호', '영수증번호', '채널구분', '제휴사', '점포코드', '상품코드', '구매일자', '방문일자', '구매시간', '구매금액', '구매수량', '타입']]
lpay = lpay[['고객번호', '영수증번호', '채널구분', '제휴사', '점포코드', '상품코드', '구매일자', '방문일자', '구매시간', '구매금액', '구매수량', '타입']]

# 데이터 타입 변경
lpay['영수증번호'] = lpay['영수증번호'].astype('object')
lpay['점포코드'] = lpay['점포코드'].astype('object')
cop['상품코드'] = cop['상품코드'].astype('object')
lpay['상품코드'] = lpay['상품코드'].astype('object')
pdde['구매금액'] = pdde['구매금액'].astype('int')
pdde['방문일자'] = pdde['방문일자'].astype('int64')
lpay['방문일자'] = lpay['방문일자'].astype('int64')

In [8]:
# 데이터 concat
df = pd.concat([pdde,cop,lpay])
# 고객 정보 테이븛 통합
df = pd.merge(df, demo, on = ['고객번호'], how = 'left')
# 상품 분류 테이블 통합
df = pd.merge(df, br, on = ['점포코드','제휴사'], how = 'left')
# 점포 정보 테이블 통합
df = pd.merge(df, clac, on = ['상품코드'], how = 'left')
# 코로나 데이터와 병합을 위하여 날짜열 생성
df['날짜'] = df['구매일자'].astype(str)
df['날짜'] = pd.to_datetime(df['날짜'])
# 코로나 외부 데이터 통합
df = pd.merge(df, covid19, on = ['날짜'], how = 'left')

In [9]:
# 코로나 확진자수의 데이터가 2021년 부터 존재하기 때문에 2020년의 5일간의 데이터는 2021년 1월 1일의 데이터 값으로 대체
df['신규확진자'].fillna(824, inplace = True)
df['신규사망자'].fillna(25, inplace = True)

In [10]:
# 결측치 확인 -> 데이터를 합치면서 생기는 결측치만 존재
df.isna().sum()

고객번호             0
영수증번호            0
채널구분             0
제휴사              0
점포코드        353184
상품코드        601488
구매일자             0
구매시간             0
구매금액             0
구매수량             0
타입               0
방문일자             0
성별               0
연령대              0
거주지대분류코드         0
점포대분류코드     729023
점포중분류코드     729023
소분류명        601488
대분류명        601488
중분류명        601488
날짜               0
신규확진자            0
신규사망자            0
dtype: int64

In [11]:
# 알 수 없는 정보의 결측치를 없음으로 채움
df.fillna('없음', inplace = True)

In [12]:
# 데이터 저장
df.to_csv('../data/01_Data_Preprocess.csv', index = False)

In [13]:
# 데이터가 제대로 저장되었는지 확인
pd.read_csv('../data/01_Data_Preprocess.csv')

,고객번호,영수증번호,채널구분,제휴사,점포코드,상품코드,구매일자,구매시간,구매금액,구매수량,...,연령대,거주지대분류코드,점포대분류코드,점포중분류코드,소분류명,대분류명,중분류명,날짜,신규확진자,신규사망자
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000,1,...,50대,Z17,Z10,Z10042,남성티셔츠,남성의류,남성의류상의,2021-01-01,824.0,25.0
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700,1,...,40대,Z13,Z13,Z13001,기타주방일회용품,주방잡화,주방일회용품,2021-01-01,824.0,25.0
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000,1,...,50대,Z17,Z10,Z10042,남성티셔츠,남성의류,남성의류상의,2021-01-01,824.0,25.0
3,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000,1,...,50대,Z17,Z10,Z10042,남성티셔츠,남성의류,남성의류상의,2021-01-01,824.0,25.0
4,M430112881,A01000005297,1,A01,A010039,PD1692,20210101,10,9900,1,...,50대,Z17,Z10,Z10042,커피/음료,테넌트/음식점,식당,2021-01-01,824.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745872,M470112503,210125110644545,2,A06,없음,없음,20210125,11,51800,0,...,50대,Z16,없음,없음,없음,없음,없음,2021-01-25,354.0,11.0
4745873,M470112503,210104210652271,2,A06,없음,없음,20210104,21,6440,0,...,50대,Z16,없음,없음,없음,없음,없음,2021-01-04,715.0,26.0
4745874,M748878049,210119200278240,1,A02,없음,없음,20210119,20,17600,0,...,30대,Z10,없음,없음,없음,없음,없음,2021-01-19,403.0,17.0
4745875,M748878049,210119180267206,1,A02,없음,없음,20210119,18,12400,0,...,30대,Z10,없음,없음,없음,없음,없음,2021-01-19,403.0,17.0
